In [6]:
import pandas as pd
import networkx as nx
import ast
import numpy as np
import seaborn as sns
from matplotlib.pylab import plt
import itertools
import community
%matplotlib inline

In [7]:
#a = pd.read_csv("./the-movies-dataset/credits.csv")
credits = pd.read_csv("./the-movies-dataset/credits.csv")
actors = pd.read_csv("./actors.csv")
metadata = pd.read_csv("./the-movies-dataset/movies_metadata.csv")
metadata_set_index = metadata.set_index('id')
actors_set_index = actors.set_index('credit_id')

/Users/user1/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
directors = pd.read_csv("./directors.csv")
directors_set_index = directors.set_index('credit_id')

In [63]:
directors_set_index.loc['52fe4943c3a368484e122b45']

SyntaxError: invalid syntax (<ipython-input-63-6e5eb790d75a>, line 1)

In [65]:
directors_set_index.loc['52fe4284c3a36847f8024f49']

Unnamed: 0                                                      0
gender                                                          2
id                                                           7879
name                                                John Lasseter
profile_path                     /7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
movies          [862, 9487, 863, 13925, 920, 49013, 13927, 139...
appearance                                                     10
Name: 52fe4284c3a36847f8024f49, dtype: object

In [60]:
directors.head(1)

,Unnamed: 0,credit_id,gender,id,name,profile_path,movies,appearance
0,0,52fe4284c3a36847f8024f49,2,7879,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,"[862, 9487, 863, 13925, 920, 49013, 13927, 139...",10


In [8]:
actors_sorted = actors.sort_values(by = ['appearance'], ascending = False)
#actors_sorted.head()

In [4]:
credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [5]:
metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


映画ごとの、キャストのリストを取得する

In [12]:
## データフレーム中のstringをキャスト(dict型)のリストに変換する
def make_list_from_str(string):
    if isinstance(string, str):
        result_list = eval(string)
    else:
        result_list = np.nan
    return result_list

In [13]:
credits.cast = credits.cast.map(make_list_from_str)
credits.crew = credits.crew.map(make_list_from_str)

In [15]:
#metadata = metadata.drop([19730,29503,35587])
metadata.id = pd.to_numeric(metadata.id)
metadata.release_date = pd.to_datetime(metadata['release_date'], format='%Y-%m-%d', errors='ignore')

映画+キャストのデータセットを新しいものに制限する

In [16]:
def cre_mov_pickup(datetime):
    c_m = pd.merge(credits, metadata, right_on ='id' ,left_on='id', how = 'left')
    c_m_sorted = c_m.sort_values(by = ['release_date'], ascending = False)
    c_m_sorted_pickup = c_m_sorted[c_m_sorted['release_date'] > pd.datetime.strptime(datetime,'%Y-%m-%d')]
    movies_cast_dict = c_m_sorted_pickup[["cast","crew"]].to_dict('record')
    return movies_cast_dict

ピックアップしたいアクターのリストを取得する

In [17]:
def make_actors_list(min_appearance):
    actors_sorted = actors.sort_values(by = ['appearance'], ascending = False)
    actors_pickup = actors_sorted[actors_sorted['appearance'] > 10]
    pickup_actors_list = actors_pickup['credit_id'].tolist()
    return pickup_actors_list

In [70]:
#キャストと映画の配列を作る（処理には使ってない）
castmoviearr = np.zeros((206158,45476))
castmovie = pd.DataFrame(castmoviearr, columns=credits["id"].tolist(), index=actors_info["credit_id"].tolist())
castmovie2 = pd.merge(actors_pickup, castmovie, right_index =True , left_on='credit_id', how = 'inner')
castmovie2.shape

共起リストを作る

In [26]:
movies_cast_dict[0]['crew']

[{'credit_id': '52fe4943c3a368484e122b45',
  'department': 'Directing',
  'gender': 2,
  'id': 2710,
  'job': 'Director',
  'name': 'James Cameron',
  'profile_path': '/zy2foCd8PEtvCcsX48cROdQdDLB.jpg'},
 {'credit_id': '52fe4943c3a368484e122b57',
  'department': 'Writing',
  'gender': 2,
  'id': 2710,
  'job': 'Screenplay',
  'name': 'James Cameron',
  'profile_path': '/zy2foCd8PEtvCcsX48cROdQdDLB.jpg'},
 {'credit_id': '57927bbac3a3684012004e85',
  'department': 'Writing',
  'gender': 2,
  'id': 2710,
  'job': 'Characters',
  'name': 'James Cameron',
  'profile_path': '/zy2foCd8PEtvCcsX48cROdQdDLB.jpg'},
 {'credit_id': '57927bc5c3a3681878002526',
  'department': 'Writing',
  'gender': 0,
  'id': 507,
  'job': 'Screenplay',
  'name': 'Josh Friedman',
  'profile_path': '/sr7fib6UVbIXQXGCSEw8L0IU6o3.jpg'},
 {'credit_id': '57927bd9c3a368286d001b1a',
  'department': 'Writing',
  'gender': 2,
  'id': 6039,
  'job': 'Writer',
  'name': 'Shane Salerno',
  'profile_path': None},
 {'credit_id': 

In [67]:
def makekyoukilist(movies_cast_dict, pickup_actors_list):
    castkyoukilist = []
    for movie in movies_cast_dict:
        director = []
        castlist = []
        crewlist = []
        cast_dict = movie['cast']
        crew_dict = movie['crew']
        for cast in cast_dict:
            if cast['credit_id'] in pickup_actors_list:
                castlist.append(cast['credit_id'])
        for crew in crew_dict:
            if crew['job'] == 'Director':
                director.append(crew['credit_id'])
            else:
                crewlist.append(crew['credit_id'])
        castkyoukilist.append({'director':director, 'castlist':castlist, 'crewlist':crewlist})
    return castkyoukilist

アクターの情報を取得する関数

In [28]:
def get_actor_tag(creditid, tagname):
    tagvalue = actors_set_index.loc[creditid][tagname]
    #print(type(tagvalue))
    return tagvalue

In [34]:
def get_director_tag(creditid, tagname):
    tagvalue = directors_set_index.loc[creditid][tagname]
    return tagvalue

ネットワーク化する

In [75]:
def makeGraph(castkyoukilist):
    G_cast = nx.Graph()
    G_crew = nx.Graph()
    for movie in castkyoukilist:
        director = movie['director']
        print(type(director))
        castlist = movie['castlist']
        crewlist = movie['crewlist']
        addDirector(G_cast,director)
        addNodes(G_cast, castlist)
        addEdges(G_cast, director, castlist)
        addDirector(G_crew,director)
        addNodes_crew(G_crew, crewlist)
        addEdges(G_crew, director, crewlist)
    return G_cast, G_crew


def addDirector(G,director):
    try:
        G.add_node(director, attr_dict = {'d':1,'name':get_director_tag(director,'name'),'appearance':int(get_director_tag(director,'appearance'))})
    except:
        print('error : ' + director)

def addNodes_crew(G,crewlist):
    for crew in crewlist:
        G.add_node(crew, attr_dict = {'d':0})
        
def addNodes(G,castlist):
    for cast in castlist:
        #G.add_node(get_actor_tag(cast,'name'), size = get_actor_tag(cast,'appearance'))
        G.add_node(cast, attr_dict = {'d':0,'name':get_actor_tag(cast,'name'), 'appearance':int(get_actor_tag(cast,'appearance'))})

def addEdges(G, director, castlist):
    for cast in castlist:
        if G.has_edge(director, cast):
            G.edge[director][cast]["weight"] += 1
        else:
            G.add_edge(director, cast, {"weight":1})
    

In [33]:
def clusteringplot(G,partition):                                                                                                           
    size = float(len(set(partition.values())))                                                                                                 
    pos = nx.spring_layout(G)                                                                                                                  
    count = 0.                                                                                                                                 
    for com in set(partition.values()):                                                                                                        
        count += 1.                                                                                                                              
        list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com] 
        #node_size = [ d["count"]*20 for (n,d) in G.nodes(data=True)]
        nx.draw_networkx_nodes(G, pos, list_nodes, node_color = str(count/size), node_size=30)   
        nx.draw_networkx_labels(G, pos, fontsize=14, font_family="Yu Gothic", font_weight="bold")
    plt.show()

In [19]:
def dividecluster(partition):
    valuelist = list(set(partition.values()))
    num = len(valuelist)
    arr = [[] for i in range(num)]
    for k,v in partition.items():
        arr[v].append(k)
    return arr

全て実行し、グラフを出力

In [38]:
pickup_actors_list = make_actors_list(1)
movies_cast_dict = cre_mov_pickup('1970-01-01')

In [57]:
kyoukilist = makekyoukilist(movies_cast_dict, pickup_actors_list)

In [77]:
directors = []
for movie in kyoukilist:
    directors.append(movie['director'])

In [78]:
len(directors)

37116

In [76]:
G_cast, G_crew = makeGraph(kyoukilist)

<class 'str'>
error : 52fe4943c3a368484e122b45
error : 52fe4943c3a368484e122b45
<class 'str'>
error : 54516d4b0e0a2601d8004b8a
error : 54516d4b0e0a2601d8004b8a
<class 'str'>
error : 59c81784c3a368144306370c
error : 59c81784c3a368144306370c
<class 'str'>
error : 55c5838e9251417a2a000d09
error : 55c5838e9251417a2a000d09
<class 'str'>
<class 'str'>
error : 5461f666c3a3686f4c001f27
error : 5461f666c3a3686f4c001f27
<class 'str'>
error : 53f7f9b1c3a368735e000607
error : 53f7f9b1c3a368735e000607
<class 'str'>
error : 5431de49c3a36825d300007e
error : 5431de49c3a36825d300007e
<class 'str'>
error : 556242cec3a368208b002eb6
error : 556242cec3a368208b002eb6
<class 'str'>
error : 5847033992514166ba023648
error : 5847033992514166ba023648
<class 'str'>
<class 'str'>
error : 57cb45dfc3a3685a530061d7
error : 57cb45dfc3a3685a530061d7
<class 'str'>
error : 535e68b90e0a264fe10065f1
error : 535e68b90e0a264fe10065f1
<class 'str'>
error : 59230a41c3a3687a4b06903a
error : 59230a41c3a3687a4b06903a
<class 'str'

error : 58fa92ed92514163df00308f
error : 58fa92ed92514163df00308f
<class 'str'>
<class 'str'>
<class 'str'>
error : 58e54c9c92514128100303b4
error : 58e54c9c92514128100303b4
<class 'str'>
<class 'str'>
error : 58e1e2529251412844008b39
error : 58e1e2529251412844008b39
<class 'str'>
<class 'str'>
<class 'str'>
error : 55da1a29c3a3683207005312
error : 55da1a29c3a3683207005312
<class 'str'>
<class 'str'>
error : 5712a00e9251410aab003cf5
error : 5712a00e9251410aab003cf5
<class 'str'>
error : 58dd3a2f92514111fb0000e2
error : 58dd3a2f92514111fb0000e2
<class 'str'>
<class 'str'>
error : 58dd29dfc3a368223e00c925
error : 58dd29dfc3a368223e00c925
<class 'str'>
error : 568c3b0492514131df02c139
error : 568c3b0492514131df02c139
<class 'str'>
<class 'str'>
<class 'str'>
error : 55c570379251417a21000b54
error : 55c570379251417a21000b54
<class 'str'>
error : 579b15fa925141543d002f13
error : 579b15fa925141543d002f13
<class 'str'>
error : 5882c18bc3a368064d002f84
error : 5882c18bc3a368064d002f84
<class '

error : 560b8b90c3a36868140051a3
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
error : 57b6b1a5c3a3683f000001e9
error : 57b6b1a5c3a3683f000001e9
<class 'str'>
error : 556c9afe9251411a42003e9c
error : 556c9afe9251411a42003e9c
<class 'str'>
error : 5327f970c3a3683179001eb4
error : 5327f970c3a3683179001eb4
<class 'str'>
error : 5858c560c3a368315006006d
error : 5858c560c3a368315006006d
<class 'str'>
error : 577a238fc3a3686e98000183
error : 577a238fc3a3686e98000183
<class 'str'>
<class 'str'>
<class 'str'>
error : 5858bcc4c3a368258f062855
error : 5858bcc4c3a368258f062855
<class 'str'>
error : 58892b02c3a36859e6012bca
error : 58892b02c3a36859e6012bca
<class 'str'>
error : 52fe4d3ec3a36847f8256739
error : 52fe4d3ec3a36847f8256739
<class 'str'>
error : 56979c3bc3a3683aea002498
error : 56979c3bc3a3683aea002498
<class 'str'>
error : 5714ef3f92514146c80003ea
error : 5714ef3f92514146c80003ea
<class 'str'>
error : 5836ef3bc3a368247d00c56e
error : 5836ef3bc3a368247d00c56e
<class 'str'>
err

<class 'str'>
<class 'str'>
error : 57a730bf92514117bf00096f
error : 57a730bf92514117bf00096f
<class 'str'>
error : 57dc787892514179d8000b57
error : 57dc787892514179d8000b57
<class 'str'>
error : 56b27d6bc3a36845ba000d7f
error : 56b27d6bc3a36845ba000d7f
<class 'str'>
<class 'str'>
error : 56ea8f2dc3a36815f1001cb4
error : 56ea8f2dc3a36815f1001cb4
<class 'str'>
error : 57057c4bc3a3680dd400023d
error : 57057c4bc3a3680dd400023d
<class 'str'>
<class 'str'>
error : 57c194bcc3a3684bbf009ab6
error : 57c194bcc3a3684bbf009ab6
<class 'str'>
<class 'str'>
<class 'str'>
error : 55b154ff92514125da00ebd8
error : 55b154ff92514125da00ebd8
<class 'str'>
error : 53da6ee40e0a2652e8002973
error : 53da6ee40e0a2652e8002973
<class 'str'>
<class 'str'>
<class 'str'>
error : 54cd5be79251415cb8000d4a
error : 54cd5be79251415cb8000d4a
<class 'str'>
<class 'str'>
error : 555934679251415576000332
error : 555934679251415576000332
<class 'str'>
error : 56801d9e92514132db00b589
error : 56801d9e92514132db00b589
<class '

<class 'str'>
error : 553e782892514137f90039c7
error : 553e782892514137f90039c7
<class 'str'>
error : 52fe4eb5c3a36847f82a2d2f
error : 52fe4eb5c3a36847f82a2d2f
<class 'str'>
<class 'str'>
error : 5734cec8c3a36874de000241
error : 5734cec8c3a36874de000241
<class 'str'>
error : 55948f769251415a0f000870
error : 55948f769251415a0f000870
<class 'str'>
<class 'str'>
error : 57413537c3a36807200045b1
error : 57413537c3a36807200045b1
<class 'str'>
error : 574203fd9251416ce60009ca
error : 574203fd9251416ce60009ca
<class 'str'>
<class 'str'>
<class 'str'>
error : 56816a3ec3a3684be9010706
error : 56816a3ec3a3684be9010706
<class 'str'>
<class 'str'>
error : 573d31209251413678000d91
error : 573d31209251413678000d91
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
error : 5680097792514132db00b213
error : 5680097792514132db00b213
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
error : 5697b5e792514154d2002526
error : 5697b5e792514154d2002526
<class 'str'>
<class 'str'>
erro

...

error : 52fe4808c3a36847f8154f25
<class 'str'>
error : 52fe4593c3a36847f80ceee9
error : 52fe4593c3a36847f80ceee9
<class 'str'>
error : 52fe45b3c3a368484e06a1a7
error : 52fe45b3c3a368484e06a1a7
<class 'str'>
error : 52fe4a58c3a36847f81c896d
error : 52fe4a58c3a36847f81c896d
<class 'str'>
error : 5361171d0e0a2652a2000048
error : 5361171d0e0a2652a2000048
<class 'str'>
<class 'str'>
error : 52fe4cf0c3a368484e1ce109
error : 52fe4cf0c3a368484e1ce109
<class 'str'>
<class 'str'>
error : 52fe492cc3a368484e11e147
error : 52fe492cc3a368484e11e147
<class 'str'>
error : 52fe45a3c3a36847f80d231f
error : 52fe45a3c3a36847f80d231f
<class 'str'>
error : 52fe45a3c3a36847f80d231f
error : 52fe45a3c3a36847f80d231f
<class 'str'>
error : 52fe43cac3a36847f8070003
error : 52fe43cac3a36847f8070003
<class 'str'>
error : 52fe45669251416c75055153
error : 52fe45669251416c75055153
<class 'str'>
error : 52fe4637c3a368484e084ce9
error : 52fe4637c3a368484e084ce9
<class 'str'>
<class 'str'>
<class 'str'>
error : 52fe45759

<class 'str'>
error : 52fe44479251416c9100b233
error : 52fe44479251416c9100b233
<class 'str'>
<class 'str'>
error : 52fe45f8c3a36847f80e5095
error : 52fe45f8c3a36847f80e5095
<class 'str'>
<class 'str'>
error : 58b34944c3a36852a300b54c
error : 58b34944c3a36852a300b54c
<class 'str'>
error : 52fe45fec3a368484e07aa89
error : 52fe45fec3a368484e07aa89
<class 'str'>
<class 'str'>
error : 52fe47c2c3a368484e0d804f
error : 52fe47c2c3a368484e0d804f
<class 'str'>
<class 'str'>
error : 52fe4408c3a36847f807e389
error : 52fe4408c3a36847f807e389
<class 'str'>
error : 551473bc9251412be4001953
error : 551473bc9251412be4001953
<class 'str'>
error : 52fe4944c3a36847f8191b05
error : 52fe4944c3a36847f8191b05
<class 'str'>
error : 52fe48d59251416c750b4adb
error : 52fe48d59251416c750b4adb
<class 'str'>
<class 'str'>
error : 52fe4508c3a36847f80b865d
error : 52fe4508c3a36847f80b865d
<class 'str'>
<class 'str'>
error : 52fe439b9251416c7501701b
error : 52fe439b9251416c7501701b
<class 'str'>
error : 52fe457a925141

error : 52fe442dc3a36847f8086d21
<class 'str'>
error : 52fe484bc3a36847f816007d
error : 52fe484bc3a36847f816007d
<class 'str'>
<class 'str'>
error : 52fe48559251416c91088957
error : 52fe48559251416c91088957
<class 'str'>
error : 52fe4d38c3a36847f8255191
error : 52fe4d38c3a36847f8255191
<class 'str'>
error : 52fe43fbc3a36847f807ba83
error : 52fe43fbc3a36847f807ba83
<class 'str'>
error : 52fe4297c3a36847f802ab35
error : 52fe4297c3a36847f802ab35
<class 'str'>
error : 52fe47879251416c7509caeb
error : 52fe47879251416c7509caeb
<class 'str'>
error : 52fe487bc3a36847f8169247
error : 52fe487bc3a36847f8169247
<class 'str'>
error : 52fe48199251416c910802f7
error : 52fe48199251416c910802f7
<class 'str'>
error : 52fe44b29251416c7503dc49
error : 52fe44b29251416c7503dc49
<class 'str'>
<class 'str'>
<class 'str'>
error : 52fe47c99251416c750a5741
error : 52fe47c99251416c750a5741
<class 'str'>
error : 52fe4a33c3a36847f81c067b
error : 52fe4a33c3a36847f81c067b
<class 'str'>
error : 52fe4617c3a36847f80ec51

error : 52fe4b49c3a368484e18176b
error : 52fe4b49c3a368484e18176b
<class 'str'>
error : 5506126e92514153470019bf
error : 5506126e92514153470019bf
<class 'str'>
error : 52fe4e4e9251416c9112d503
error : 52fe4e4e9251416c9112d503
<class 'str'>
error : 52fe4a79c3a368484e1554eb
error : 52fe4a79c3a368484e1554eb
<class 'str'>
error : 52fe4606c3a36847f80e7e4d
error : 52fe4606c3a36847f80e7e4d
<class 'str'>
error : 52fe46fec3a368484e0af51d
error : 52fe46fec3a368484e0af51d
<class 'str'>
<class 'str'>
error : 52fe48f4c3a36847f8182173
error : 52fe48f4c3a36847f8182173
<class 'str'>
error : 52fe44919251416c750394cf
error : 52fe44919251416c750394cf
<class 'str'>
error : 52fe4952c3a368484e12585b
error : 52fe4952c3a368484e12585b
<class 'str'>
error : 52fe49f6c3a36847f81b03f3
error : 52fe49f6c3a36847f81b03f3
<class 'str'>
error : 52fe45e3c3a368484e074b17
error : 52fe45e3c3a368484e074b17
<class 'str'>
error : 538f1dae0e0a2644ad001796
error : 538f1dae0e0a2644ad001796
<class 'str'>
<class 'str'>
error : 52fe

error : 52fe4431c3a368484e014be9
<class 'str'>
<class 'str'>
error : 52fe441cc3a36847f8082ca7
error : 52fe441cc3a36847f8082ca7
<class 'str'>
error : 52fe43dfc3a36847f807568b
error : 52fe43dfc3a36847f807568b
<class 'str'>
error : 52fe4707c3a36847f811fa35
error : 52fe4707c3a36847f811fa35
<class 'str'>
error : 52fe45789251416c91033707
error : 52fe45789251416c91033707
<class 'str'>
error : 52fe4608c3a36847f80e8655
error : 52fe4608c3a36847f80e8655
<class 'str'>
<class 'str'>
<class 'str'>
error : 52fe46ef9251416c91064a61
error : 52fe46ef9251416c91064a61
<class 'str'>
error : 52fe4a33c3a36847f81c0523
error : 52fe4a33c3a36847f81c0523
<class 'str'>
error : 52fe4a46c3a36847f81c4bf1
error : 52fe4a46c3a36847f81c4bf1
<class 'str'>
<class 'str'>
<class 'str'>
error : 52fe43619251416c7500e0f1
error : 52fe43619251416c7500e0f1
<class 'str'>
<class 'str'>
<class 'str'>
error : 52fe47f6c3a368484e0e2bcb
error : 52fe47f6c3a368484e0e2bcb
<class 'str'>
error : 52fe494e9251416c750c3b9f
error : 52fe494e925141

In [154]:
print(G.number_of_nodes())
print(G.number_of_edges())

6778
9920


In [151]:
nx.write_gexf(G, "casts_5.gexf")

その他の処理(クラスタリングとか)

In [49]:
partition = community.best_partition(G)
#clusteringplot(G,partition)

#ノードに名前つけるのと、共演回数でエッジの重みを変えるのがまだ

In [50]:
arr = dividecluster(partition)
with open('clustering.txt','w') as f:
    for cluster in arr:
        print(arr.index(cluster))
        print(cluster)

0
['52fe4284c3a36847f8024f95', '52fe4284c3a36847f8024f99', '52fe4284c3a36847f8024f9d', '52fe4284c3a36847f8024fa5', '52fe4284c3a36847f8024fa9', '52fe4284c3a36847f8024fb5', '52fe4284c3a36847f8024fb9']
1
['52fe44bfc3a36847f80a7c73', '52fe44bfc3a36847f80a7c77', '52fe44bfc3a36847f80a7c9d', '52fe44c0c3a36847f80a7ceb', '52fe44c0c3a36847f80a7cef', '52fe44bfc3a36847f80a7ca1']
2
['52fe466a9251416c75077a8d', '52fe466a9251416c75077a91', '52fe466a9251416c75077a95', '52fe466a9251416c75077a99', '52fe466a9251416c75077a9d', '53e5fcc2c3a3684430000d65', '53e5fcd4c3a3684433000e1a']
3
['52fe44779251416c91011ab1', '52fe44779251416c91011ab5', '52fe44779251416c91011ac1', '52fe44779251416c91011ac5', '52fe44779251416c91011acf', '56f8a929c3a36816e80084f3']
4
['52fe44959251416c75039eb9', '52fe44959251416c75039ebd', '52fe44959251416c75039ec1', '52fe44959251416c75039ef3', '52fe44959251416c75039ef7', '52fe44959251416c75039efb', '52fe44959251416c75039f07', '52fe44959251416c75039f0b']
5
['52fe4292c3a36847f80291f5', '5

422
['52fe454e9251416c75052035']
423
['52fe43729251416c75010a47']
424
['52fe43f29251416c75023d3b']
425
['52fe45119251416c9102661d']
426
['55104902c3a368487400dca3']
427
['52fe44ac9251416c7503cfb9', '56888fa2c3a3684be9023f67', '56888fcac3a36860e90252e9', '56889001c3a3684be9023f76', '56889086c3a3684be9023f90', '5688910fc3a3684be9023f9e', '547a51f49251414c65000478']
428
['5791d9a79251415fd8005657']
429
['52fe463d9251416c75071d99', '52fe463d9251416c75071da9', '52fe463d9251416c75071db1']
430
['52fe44419251416c7502e405']
431
['52fe4810c3a36847f8156755', '5414221e0e0a26198a0018f7']
432
['52fe44879251416c7503815b', '52fe44889251416c7503815f', '52fe44889251416c75038183']
433
['52fe44819251416c9101324d', '52fe44819251416c91013255']
434
['52fe45d7c3a36847f80dd903']
435
['52fe4857c3a36847f8162977']
436
['52fe4472c3a368484e02377b']
437
['52fe47369251416c75091835']
438
['52fe43cb9251416c7501e823']
439
['54f94dc9c3a36878e1000f55']
440
['59007eb792514161a600b044', '59007e5992514161be00a3ae', '59007e47

['52fe442ac3a368484e01358b']
1195
['52fe444b9251416c7502fc07']
1196
['54624db8c3a36808200000bd']
1197
['5452be01c3a368092700029a']
1198
['52fe4b489251416c750fdf3b']
1199
['52fe46479251416c9104f4b9', '57c5fb2f925141214d000338']
1200
['52fe456f9251416c7505673d', '52fe456f9251416c75056741', '557fbb70925141256b000a1f']
1201
['52fe44e2c3a36847f80afcd3', '52fe44e2c3a36847f80afcdb', '52fe44e2c3a36847f80afc91', '57538e0dc3a3680e820015c7', '53ba7bfbc3a368072e00405d']
1202
['52fe4982c3a368484e12f343']
1203
['52fe4530c3a368484e04d25d']
1204
['52fe44e2c3a368484e03c86d', '52fe44e2c3a368484e03c871']
1205
['52fe44869251416c75037cd5']
1206
['52fe425ac3a36847f8017873', '52fe425ac3a36847f8017897', '55e4f58d92514137a2000552', '55e502d1c3a36813ed00090e']
1207
['52fe482dc3a36847f815b503', '52fe482dc3a36847f815b50f']
1208
['52fe43e2c3a368484e003b0d', '56739bed9251412dab0011c6']
1209
['52fe4310c3a36847f80378d7']
1210
['52fe451bc3a36847f80bcd0b', '52fe451bc3a36847f80bcd13']
1211
['52fe4275c3a36847f8020269']
1

Cytoscapeか何かで可視化する用

networkxのHPに大体のできることが書いてある
https://networkx.github.io/documentation/networkx-1.9/reference/functions.html

In [18]:
labels = dict([(i, str(i)) for i in range(nx.number_of_nodes(G))])
nx.set_node_attributes(G, 'community', partition)
#nx.set_node_attributes(G, 'label', labels)
nx.write_gml(G, "community.gml")

NameError: name 'partition' is not defined